In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Lets check your GPU
if tf.config.list_logical_devices('GPU') != []:
    print("LET'S DO THISS!!! :DDDD")
else: 
    print("Oh dude... D:")

LET'S DO THISS!!! :DDDD


In [2]:
with open('data/input.txt') as f:
    content = f.read().lower()
    
print(content[:100])

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [3]:
"".join(sorted(set(content.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [4]:
text_vec_layer = keras.layers.TextVectorization(
                                                split="character",
                                                standardize="lower"
                                            )
text_vec_layer.adapt([content])
encoded = text_vec_layer([content])[0]

In [5]:
def decoder(seq, delta=2):
    output = np.array(text_vec_layer.get_vocabulary())[seq + delta]
    new_output = []
    for i in range(output.shape[0]):
        phrase = ""
        for j in range(output.shape[1]):
            phrase += output[i, j]
        new_output.append(phrase)
    return np.array(new_output)

In [7]:
loaded_model = keras.models.load_model('my_shakespeare_model')

In [23]:
decoder(text_vec_layer(['To be or not t']), delta=0)[:, np.newaxis]

array([['to be or not t']], dtype='<U14')

In [24]:
len('To be or not t')

14

In [35]:
text_vec_layer(['To be or '])

<tf.Tensor: shape=(1, 9), dtype=int64, numpy=array([[ 4,  5,  2, 23,  3,  2,  5, 10,  2]], dtype=int64)>

In [44]:
tf.argmax(loaded_model.predict(text_vec_layer(['To be or '])), axis=2)

1/1 [==============================] - 0s 29ms/step


<tf.Tensor: shape=(1, 9), dtype=int64, numpy=array([[ 9,  9,  6,  0,  0,  6, 25,  2,  6]], dtype=int64)>

In [43]:
text_vec_layer.get_vocabulary()[6 + 2]

'h'

In [51]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    loaded_model
])

In [52]:
shakespeare_model = shakespeare_model
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [53]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [54]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU


In [55]:
print(extend_text("To be or not to be", temperature=0.01))

1/1 [==============================] - 0s 41ms/step
To be or not to be a strange and the duke i shall be the duke i shal
